In [1]:
import pandas as pd
import sys
import requests as r

sys.path.append('../helper_functions')
import github_utils as gh
import tokenlist_utils as tl
import duneapi_utils as dapi

url_content = [
    # owner_name, repo_name, path_name, file_name
      ['ethereum-optimism','ethereum-optimism.github.io','','optimism.tokenlist.json',] #OP Bridge - https://github.com/ethereum-optimism/ethereum-optimism.github.io/blob/master/optimism.tokenlist.json
    , ['rainbow-me','rainbow-token-list','output','lean-rainbow-token-list.json'] #Rainbow - https://github.com/rainbow-me/rainbow-token-list/blob/main/output/lean-rainbow-token-list.json
    , ['velodrome-finance','docs','public','tokenlist.json'] #Velo - https://github.com/velodrome-finance/docs/blob/aa557db47248b88af73dcdebfbc161e2b93da712/public/tokenlist.json#L4
]

In [2]:
dune_list_query_id = 2420477 #https://dune.com/queries/2420477

In [3]:
# Load the JSON data from the URL
df_list = []
for gh_url in url_content:

    owner_name = gh_url[0]
    repo_name = gh_url[1]
    path_name = gh_url[2]
    file_name = gh_url[3]

    gh_file = gh.get_file_url_from_github(owner_name,repo_name,path_name,file_name)
    res = r.get(gh_file)
    data = res.json()
    tmp = tl.generate_table_from_tokenlist(data)
    tmp['list_name'] = owner_name
    df_list.append(tmp)

df = pd.concat(df_list)
df = df[df['chainId'] ==10] #Get OP Version

df = df.groupby(['chainId', 'address', 'name', 'symbol', 'decimals'])['list_name'].agg(list)
df = df.reset_index()
display(df)

,chainId,address,name,symbol,decimals,list_name
0,10,0x00F932F0FE257456b32dedA4758922E56A4F4b42,Paper,PAPER,18,[ethereum-optimism]
1,10,0x00a35FD824c717879BF370E70AC6868b95870Dfb,Iron Bank,IB,18,[velodrome-finance]
2,10,0x01B8b6384298D4848E3BE63D4C9D17830EeE488A,DAOhaus Token on Optimism,HAUS,18,[ethereum-optimism]
3,10,0x09448876068907827ec15F49A8F1a58C70b04d45,Ripae Share ETH,sETHo,18,[velodrome-finance]
4,10,0x0994206dfe8de6ec6920ff4d779b0d950605fb53,Curve DAO Token,CRV,18,[ethereum-optimism]
...,...,...,...,...,...,...
185,10,0xef6301da234fc7b0545c6e877d3359fe0b9e50a4,SUKU,SUKU,18,[ethereum-optimism]
186,10,0xf98dcd95217e15e05d8638da4c91125e59590b07,Kromatika,KROM,18,[ethereum-optimism]
187,10,0xfD389Dc9533717239856190F42475d3f263a270d,Granary Token,GRAIN,18,[velodrome-finance]
188,10,0xfDeFFc7Ad816BF7867C642dF7eBC2CC5554ec265,Beefy Velo,BeVELO,18,[velodrome-finance]


In [4]:
dune_list = dapi.get_dune_data(dune_list_query_id)
# display(dune_list)

2023-07-21 09:06:37.595 | INFO     | duneapi_utils:get_dune_data:41 - Results available at https://dune.com/queries/2420477
2023-07-21 09:06:38.456 | INFO     | duneapi_utils:get_dune_data:86 - ✨ Results saved as csv_outputs/my_query_results.csv, with 517 rows and 4 columns.


In [5]:
#token addresses to lower
df['contract_address'] = df['address'].str.lower().str.strip()
dune_list['contract_address'] = dune_list['contract_address'].str.lower().str.strip()

In [6]:
adds = df[~df['contract_address'].isin(dune_list['contract_address'])]
display(adds[['address','symbol','decimals', 'name','list_name']])


,address,symbol,decimals,name,list_name
90,0x7d14206c937e70e19e3a5b94011faf0d5b3928e2,VITA,18,VitaDAO Token,[ethereum-optimism]
114,0xA8d83Cd5102b0A412931B12255D218B0b0E7152C,OPL,18,OP Launcher,[velodrome-finance]
122,0xBB6BBaa0F6D839A00c82B10747aBc3b7E2eEcc82,IBEX,18,Impermax,[ethereum-optimism]
123,0xC27D9BC194a648fE3069955a5126699C4e49351C,AMKT,18,Alongside Crypto Market Index,[ethereum-optimism]
125,0xC5B3AC2DF8D8D7AC851F763a5b3Ff23B4A696d59,APT,8,Aptos,[ethereum-optimism]
128,0xCa98d642E158c3016E1c75dbcC3D5ECFe3E02A19,ETS WETH/USDC,6,ETS Night over Arrakis,[velodrome-finance]
133,0xDeBa4568ff4CC84B420bcbFb228De5b7F2807a5c,CHAM,18,Champion,[velodrome-finance]
155,0xb1aE0a34DC3976d98C1741C92b798dAF8E0e1e11,TRX,6,TRON,[ethereum-optimism]
157,0xba1Cf949c382A32a09A17B2AdF3587fc7fA664f1,SOL,9,Solana,[ethereum-optimism]
184,0xeAeAdAC73baaF4cB8B024dE9D65B2eeFa722856C,RFWSTETH,18,Respawn Finance Wrapped Staked Ethereum,[ethereum-optimism]
